In [ ]:
# default_exp cli_colors

# Docker CLI Colors

> Provides a colorful CLI for basic Docker commands

In [ ]:
#hide
from nbverbose.showdoc import *

In [ ]:
#export
from rich import print as rprint
from rich.console import Console
from fastcore.script import call_parse, Param, store_true

In [ ]:
#export
console = Console()

In [ ]:
#export
def _run(cmd): 
    out = subprocess.run(cmd, capture_output=True, text=True)
    return out.stdout

## Rich

[Rich](https://github.com/willmcgugan/rich) is a library designed to enahnce the printing of python applications, such as below:

In [ ]:
rprint("Hello, [bold magenta]World[/bold magenta]!", ":vampire:", "TestMe")

Hello, World! 🧛 TestMe

We will use it to take very basic-looking `docker` commands, and Rich-ify them to look better

## Docker

These are the docker commands currently supported, based on personal preference of use:

- `docker search`
- `docker pull`
- `docker run`
- `docker container ls`

In [ ]:
#export
import subprocess
import json

In [ ]:
#hide
# This is an example output from `docker search fastai --format "{{json . }}"
o = '''"{"Description":"Fast.ai course 2 [Pytorch]","IsAutomated":"false","IsOfficial":"false","Name":"paperspace/fastai","StarCount":"20"}"
"{"Description":"","IsAutomated":"false","IsOfficial":"false","Name":"spellrun/fastai","StarCount":"7"}"
"{"Description":"GPU-enabled Jupyter environment for fast.ai …","IsAutomated":"true","IsOfficial":"false","Name":"deeprig/fastai-course-1","StarCount":"6"}"
"{"Description":"Official fast.ai image for fastai","IsAutomated":"false","IsOfficial":"false","Name":"fastdotai/fastai","StarCount":"6"}"
"{"Description":"This container has moved to https://hub.dock…","IsAutomated":"false","IsOfficial":"false","Name":"zerotosingularity/fastai2","StarCount":"2"}"
"{"Description":"A Docker container for fastai v2","IsAutomated":"false","IsOfficial":"false","Name":"seemeai/fastai2","StarCount":"2"}"
"{"Description":"This container has moved to https://hub.dock…","IsAutomated":"false","IsOfficial":"false","Name":"zerotosingularity/fastai","StarCount":"2"}"
"{"Description":"A Docker container for fastai","IsAutomated":"false","IsOfficial":"false","Name":"seemeai/fastai","StarCount":"1"}"
"{"Description":"","IsAutomated":"false","IsOfficial":"false","Name":"fastai/fastai","StarCount":"1"}"
"{"Description":"","IsAutomated":"false","IsOfficial":"false","Name":"gwelican/fastai","StarCount":"1"}"
"{"Description":"","IsAutomated":"false","IsOfficial":"false","Name":"fastai/codespaces","StarCount":"1"}"
"{"Description":"","IsAutomated":"false","IsOfficial":"false","Name":"fastdotai/fastai-course","StarCount":"0"}"
"{"Description":"","IsAutomated":"false","IsOfficial":"false","Name":"spellrun/fastai-base","StarCount":"0"}"
"{"Description":"","IsAutomated":"false","IsOfficial":"false","Name":"fastai/ubuntu","StarCount":"0"}"
"{"Description":"container for fastai code","IsAutomated":"false","IsOfficial":"false","Name":"sgrestdocker/fastai","StarCount":"0"}"
"{"Description":"","IsAutomated":"false","IsOfficial":"false","Name":"fastai/fastpages-jekyll","StarCount":"0"}"
"{"Description":"Official fast.ai image for fastai with an ed…","IsAutomated":"false","IsOfficial":"false","Name":"fastdotai/fastai-dev","StarCount":"0"}"
"{"Description":"","IsAutomated":"false","IsOfficial":"false","Name":"aychang/fastai-gpu","StarCount":"0"}"
"{"Description":"Fastai2, FastApi, Pytorch 1.7,  Pipenv, Cond…","IsAutomated":"false","IsOfficial":"false","Name":"lifefilm/fastai2_fastapi","StarCount":"0"}"
"{"Description":"","IsAutomated":"false","IsOfficial":"false","Name":"fastai/jekyll","StarCount":"0"}"
"{"Description":"","IsAutomated":"false","IsOfficial":"false","Name":"kuberlab/fastai","StarCount":"0"}"
"{"Description":"","IsAutomated":"false","IsOfficial":"false","Name":"krllerof/fastai","StarCount":"0"}"
"{"Description":"","IsAutomated":"false","IsOfficial":"false","Name":"grapenf/fastaindex","StarCount":"0"}"
"{"Description":"fastai","IsAutomated":"false","IsOfficial":"false","Name":"qooba/fastai","StarCount":"0"}"
"{"Description":"Fast AI container","IsAutomated":"false","IsOfficial":"false","Name":"snarkai/fastai","StarCount":"0"}"'''

In [ ]:
#export
from rich.table import Table

In [ ]:
#export
def _format_name(name):
    "Formats the container name to be colorized and colorblind-friendly"
    if '/' in name:
        full_name = name.split('/')
    if len(full_name) == 2:
        org = f'[#FFC20A]{full_name[0]}'
        name = f'[#0C7BDC]{full_name[1]}'
        name = f'{org}[white]/{name}'
    return name

In [ ]:
#export
def create_docker_search_table(data, **args):
    "Returns a `Table` from a docker search query"
    def _get_data(item):
        "Parses json and returns a list"
        return [
            item['Name'],
            item['Description'],
            item['StarCount'],
            item['IsOfficial'],
            item['IsAutomated']
        ]
    table = Table(show_header=True, header_style="bold #0C7BDC", show_lines=True, **args) # Change to blue
    table.add_column("Container Name", justify="left")
    table.add_column("Description", justify="left")
    table.add_column("Stars", justify="left")
    table.add_column("Official", justify="left")
    table.add_column("Automated", justify="left")
    split_data = data.split('\n')
    parsed_data = [json.loads(a[1:-1]) for a in split_data if len(a) > 0]
    for point in parsed_data:
        point['Name'] = _format_name(point['Name'])
        if point['IsAutomated'] == 'true':
            point['IsAutomated'] = "[green]:heavy_check_mark:"
        else:
            point['IsAutomated'] = ''
        if point['IsOfficial'] == 'true':
            point['IsOfficial'] = "[green]:heavy_check_mark:"
        else:
            point['IsOfficial'] = ''
        table.add_row(*_get_data(point))
    return table

Below you can see an example of a "docker_buddy" `docker search` result, utilizing Rich to improve the readability:

In [ ]:
#hide_input
create_docker_search_table(o)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Container Name            ┃ Description                    ┃ Stars ┃ Official ┃ Automated ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩
│ paperspace/fastai         │ Fast.ai course 2 [Pytorch]     │ 20    │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ spellrun/fastai           │                                │ 7     │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ deeprig/fastai-course-1   │ GPU-enabled Jupyter            │ 6     │          │ ✔         │
│                           │ environment for fast.ai …      │       │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ fastdotai/fastai          │ Official fast.ai image for     │ 6     │          │           │
│                           │ fastai                         │       │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ zerotosingularity/fastai2 │ This container has moved to    │ 2     │          │           │
│                           │ https://hub.dock…              │       │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ seemeai/fastai2           │ A Docker container for fastai  │ 2     │          │           │
│                           │ v2                             │       │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ zerotosingularity/fastai  │ This container has moved to    │ 2     │          │           │
│                           │ https://hub.dock…              │       │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ seemeai/fastai            │ A Docker container for fastai  │ 1     │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ fastai/fastai             │                                │ 1     │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ gwelican/fastai           │                                │ 1     │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ fastai/codespaces         │                                │ 1     │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ fastdotai/fastai-course   │                                │ 0     │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ spellrun/fastai-base      │                                │ 0     │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ fastai/ubuntu             │                                │ 0     │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ sgrestdocker/fastai       │ container for fastai code      │ 0     │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ fastai/fastpages-jekyll   │                                │ 0     │          │           │
├───────────────────────────┼────────────────────────────────┼───────┼──────────┼───────────┤
│ fastdotai/fastai-dev      │ Official fast.ai image for     │ 0     │          │           │
│                           │ fastai with an ed…             │       │          │           │
├───────────────────────────┼───────────────────────

In [ ]:
#export
@call_parse
def docker_search(
    TERM:Param("The name of a Docker image or namespace to search for", str),
    FILTER:Param("Filter output based on conditions provided", str)='',
    LIMIT:Param("Max number of search results (default 25)", int, default=25)=25,
    NO_TRUNC:Param("Don't truncate output", store_true)=False
):
    "Search the Docker Hub for images"
    cmd = ['docker','search']
    if len(FILTER) > 0:
        cmd += ['--filter', FILTER]
    if NO_TRUNC != False:
        cmd += ['--no-trunc']
    cmd += ['--limit', str(LIMIT)]
    cmd += [TERM, '--format', '"{{json . }}"']
    res = _run(cmd)
    table = create_docker_search_table(res)
    console.print(table)

In [ ]:
#hide
# Output from docker ps --format "{{json . }}" --no-trunc
o = '"{"Command":"\\"/usr/local/bin/repo2docker-entrypoint jupyter notebook --ip 0.0.0.0\\"","CreatedAt":"2021-12-05 12:01:31 -0500 EST","ID":"8ad1963d5ef736bfe321f0ac08bf7f906ee327c0d9402849e23dcfc476695f3a","Image":"fastdotai/nbdev","Labels":"repo2docker.ref=None,repo2docker.repo=local,repo2docker.version=2021.08.0","LocalVolumes":"0","Mounts":"/home/zach/docker_images/docker_buddy","Names":"nostalgic_golick","Networks":"bridge","Ports":"0.0.0.0:8888-\\u003e8888/tcp, :::8888-\\u003e8888/tcp","RunningFor":"2 hours ago","Size":"5.64MB (virtual 1.6GB)","State":"running","Status":"Up 2 hours"}"\n'

In [ ]:
#export
def _format_ports(port_string:str):
    "Color codes port forwarding"
    _incoming = '#FFC20A'
    _outgoing = '#0C7BDC'
    ports = port_string.split()
    first_port = ports[0].split(':')
    gateways = first_port[1].strip(',')
    gateways = gateways.split('->')
    gateways[0] = f'[{_incoming}]{gateways[0]}[/{_incoming}]'
    gateways[1] = f'[{_outgoing}]{gateways[1]}[/{_outgoing}]'
    first_port = first_port[0]+':'+'->'.join(gateways)
    
    second_port = ports[1].split(':::')[1]
    gateways = second_port.split('->')
    gateways[0] = f'[{_incoming}]{gateways[0]}[/{_incoming}]'
    gateways[1] = f'[{_outgoing}]{gateways[1]}[/{_outgoing}]'
    second_port = '::'+':'+'->'.join(gateways)
    return f'{first_port}, {second_port}'

In [ ]:
#export
def create_docker_ls_table(data, **args):
    "Returns a `Table` from a docker search query"
    def _get_data(item):
        "Parses json and returns a list"
        return [
            item['Names'],
            item['Image'],
            item['Status'],
            item['Ports']
        ]
    table = Table(show_header=True, header_style="bold #0C7BDC", show_lines=True, **args) # Change to blue
    table.add_column("Container Nickname", justify="left")
    table.add_column("Image", justify="left")
    table.add_column("Current Status", justify="left")
    table.add_column("Ports", justify="left")
    split_data = data.split('\n')
    parsed_data = [json.loads(a[1:-1]) for a in split_data if len(a) > 0]
    for point in parsed_data:
        point['Image'] = _format_name(point['Image'])
        if point['State'] == 'running':
            point['Status'] = f"[green]{point['Status']}[/green]"
        else:
            point['Status'] = f"[red]{point['Status']}[/red]"
        point['Ports'] = _format_ports(point['Ports'])
        del point['State']
        table.add_row(*_get_data(point))
    return table

Below you can see an example of our supercharged `docker ls` output:

In [ ]:
#hide_input
create_docker_ls_table(o)

┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Container Nickname ┃ Image           ┃ Current Status ┃ Ports                             ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ nostalgic_golick   │ fastdotai/nbdev │ Up 2 hours     │ 0.0.0.0:8888->8888/tcp,           │
│                    │                 │                │ :::8888->8888/tcp                 │
└────────────────────┴─────────────────┴────────────────┴───────────────────────────────────┘

In [ ]:
#export
@call_parse
def docker_container_ls():
    "List containers"
    cmd = ['docker','ps','--format','"{{json . }}"','--no-trunc']
    res = _run(cmd)
    table = create_docker_ls_table(res)
    console.print(table)

## Bash Commands
Each of the prior functionalities have bash commands associated with them as follows:
- `ds` -> `docker search` (or `docker_search`)
- `dls` -> `docker container ls` (or `docker_container_ls`)

This also helps with not overriding root Docker

Along with these, it is recommended to setup the following shortcuts in your `bash_profile`:
- `docker run` -> `dr`
- `docker pull` -> `dp`

So the naming conventions can remain similar (though there are no pretty tables/console bits for them)